In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
from FINDER_API import FINDER_API
from FINDER_test_utils import *
import os
import tqdm
import numpy as np


# Specify and load model

In [3]:
# model_path =  'best_models/tsp_2d/nrange_15_20_len_1011194/'
# model_path =  'best_models/tsp_2d/nrange_15_20_len_1011362/'
# model_path =  'best_models/tsp_2d/nrange_15_20_len_1009797/'
# model_path = 'best_models/tsp_2d/nrange_15_20_len_1011914/'

# insert function which automatically searches for best model with specified name
model_name = 'AGNN_default_len_10113392076640768'
model_name = 'AGNN_RNN_len_10117276006490983'
model_name = 'AGNN_no_helperfunction_len_10488634773857972'
model_path = f'saved_models/tsp_2d/nrange_15_20/{model_name}'

config_path = f'{model_path}/config.txt'
api = FINDER_API(config_path=config_path)

# get best checkpoint path
best_ckpt_file_path = get_best_ckpt(model_path)
# load checkpoint into Finder
api.load_model(ckpt_path=best_ckpt_file_path)

Gpu available: True
Built with cuda: True
model sucessfully restored from file


# Specify test sets

In [4]:
# data_dir = 'test_sets/synthetic_nrange_10_20_1000/'
# data_dir = 'test_sets/synthetic_nrange_15_20_1000/'
# data_dir = 'test_sets/synthetic_nrange_40_50_1000/'
# data_dir = 'valid_sets/synthetic_nrange_10_20_200/'
# data_dir = 'valid_sets/synthetic_nrange_15_20_200/'
data_dir = 'data/test_sets/tsp_min-n=15_max-n=20_num-graph=1000_type=random/'
data_dir_0 = 'data/test_sets/tsp_min-n=15_max-n=20_num-graph=1000_type=random/'
data_dir_1 = 'data/test_sets/tsp_min-n=40_max-n=50_num-graph=1000_type=random/'
data_dir_2 = 'data/test_sets/tsp_min-n=50_max-n=100_num-graph=1000_type=random/'
data_dir_3 = 'data/test_sets/tsp_min-n=100_max-n=200_num-graph=1000_type=random/'

test_dirs = [data_dir_0, data_dir_1, data_dir_2]

# Run tests

## Run standard tests

In [5]:
search_strategy = 'greedy'

In [6]:
# load test data
graph_list, fnames = prepare_testset_S2VDQN(data_dir)
# run test
lengths, solutions, sol_times = api.run_test(graph_list)
approx_ratios, mean_approx_ratio = get_approx_ratios(data_dir, fnames, lengths)
print(mean_approx_ratio)
# save test results
save_approx_ratios(data_dir, fnames, approx_ratios, model_name, suffix=search_strategy)
save_solutions(data_dir, fnames, solutions, model_name, suffix=search_strategy)
save_lengths(data_dir, fnames, lengths, model_name, suffix=search_strategy)


18it [00:00, 177.50it/s]

Loading test graphs...


1000it [00:05, 192.72it/s]
  6%|▌         | 58/1000 [00:00<00:01, 576.87it/s]

1.0521558291921438
Saving approximation ratios...


  3%|▎         | 31/1000 [00:00<00:03, 308.84it/s]

Saving solutions...


  6%|▌         | 60/1000 [00:00<00:01, 599.98it/s]

Saving solution lengths...


100%|██████████| 1000/1000 [00:01<00:00, 621.41it/s]


In [7]:
mean_approx_ratios = []
for data_dir in test_dirs[1::]:
    # load test data
    graph_list, fnames = prepare_testset_S2VDQN(data_dir)
    # run test
    lengths, solutions, sol_times = api.run_test(graph_list)
    approx_ratios, mean_approx_ratio = get_approx_ratios(data_dir, fnames, lengths)
    print(mean_approx_ratio)
    # save test results
    save_approx_ratios(data_dir, fnames, approx_ratios, model_name, suffix=search_strategy)
    save_solutions(data_dir, fnames, solutions, model_name, suffix=search_strategy)
    save_lengths(data_dir, fnames, lengths, model_name, suffix=search_strategy)

3it [00:00, 21.93it/s]

Loading test graphs...


1000it [00:35, 28.08it/s]
  6%|▌         | 56/1000 [00:00<00:01, 548.54it/s]

1.0455802136823251
Saving approximation ratios...


  4%|▎         | 37/1000 [00:00<00:02, 365.33it/s]

...

  4%|▍         | 45/1000 [00:00<00:02, 448.47it/s]

Saving solutions...


  6%|▌         | 57/1000 [00:00<00:01, 562.29it/s]

Saving solution lengths...


100%|██████████| 1000/1000 [00:01<00:00, 659.60it/s]


## Run tests with non-greedy search strategy


In [5]:
# use beam search during testing
search_strategy = 'beam_search+'
beam_width = 64
batch_size = 64

api.DQN.cfg['search_strategy'] = search_strategy
api.DQN.cfg['beam_width'] = beam_width
api.DQN.cfg['BATCH_SIZE'] = batch_size

suffix = f'{search_strategy}_{beam_width}'

In [6]:
mean_approx_ratios = []
for data_dir in test_dirs[0:1]:
    # load test data
    graph_list, fnames = prepare_testset_S2VDQN(data_dir)
    # run test
    lengths, solutions, sol_times = api.run_test(graph_list)
    approx_ratios, mean_approx_ratio = get_approx_ratios(data_dir, fnames, lengths)
    print(mean_approx_ratio)
    # save test results
    save_approx_ratios(data_dir, fnames, approx_ratios, model_name, suffix=suffix)
    save_solutions(data_dir, fnames, solutions, model_name, suffix=suffix)
    save_lengths(data_dir, fnames, lengths, model_name, suffix=suffix)

17it [00:00, 169.85it/s]

Loading test graphs...


1000it [00:05, 191.00it/s]
  6%|▋         | 63/1000 [00:00<00:01, 621.66it/s]

1.0474202141592173
Saving approximation ratios...


  4%|▎         | 35/1000 [00:00<00:02, 343.27it/s]

Saving solutions...


  6%|▋         | 64/1000 [00:00<00:01, 633.04it/s]

Saving solution lengths...


100%|██████████| 1000/1000 [00:01<00:00, 616.50it/s]


# Test helper function

In [ ]:
graph = graph_list[0]
length, solution, time_ = dqn.Evaluate(g=graph)

In [ ]:
solution, length

In [ ]:
# check whether length calculation is correct
def calc_tour_length(graph, solution):
    tot_len = 0
    for i in range(np.array(solution).shape[0]):
        if i == np.array(solution).shape[0] - 1:
            tot_len += graph[solution[i]][solution[0]]['weight']
        else:
            tot_len += graph[solution[i]][solution[i + 1]]['weight']
    return tot_len

In [ ]:
def add_node(graph, cur_sol, new_node):
    cur_dist = 10000000.0
    for i in range(0, np.array(cur_sol).shape[0]):
        if i + 1 == np.array(cur_sol).shape[0]:
            adj = cur_sol[0]
        else:
            adj = cur_sol[i + 1]
        if cur_sol[i] == adj:
            cost = graph[new_node][cur_sol[i]]['weight'] + graph[new_node][adj]['weight']
        else:
            cost = graph[new_node][cur_sol[i]]['weight'] + graph[new_node][adj]['weight'] - graph[cur_sol[i]][adj]['weight']
        if cost < cur_dist:
            cur_dist = cost
            pos = i + 1
    cur_sol.insert(pos, new_node)
    return  cur_dist

In [ ]:
def calc_helper_tour(g):
    cur_sol = [0]
    rem_nodes = list(np.arange(g.number_of_nodes()))
    rem_nodes.remove(0)
    tour_length = 0
    while np.array(cur_sol).shape[0] < g.number_of_nodes():
        if np.array(rem_nodes).shape[0]-1 == 0:
            rand_idx = 0
        else:
            rand_idx = np.random.randint(0, np.array(rem_nodes).shape[0]-1)
        new_node = rem_nodes[rand_idx]
        tour_length += add_node(g, cur_sol, new_node)
        rem_nodes.remove(new_node)
    return tour_length, cur_sol


In [ ]:
max_result = -np.inf
min_result = np.inf
for i in range(100):
    help_lengths = []
    for graph in graph_list:
        help_lengths.append(calc_helper_tour(graph)[0])

    true_lengths = None
    with open(data_dir+'lengths.txt', 'r') as f:
        lines = f.readlines()
        lines = [float(line.split(':')[-1].strip()) for line in lines]
    true_lengths = lines
    result = np.mean([length[0]/length[1] for length in zip(help_lengths, true_lengths)])
    if result < min_result:
        min_result = result
    if result > max_result:
        max_result = result

In [ ]:
print(min_result)
print(max_result)